### evaluate the model using the pre-trained model

In [8]:
import os, sys
import numpy as np
import tensorflow as tf
import datetime
import pandas as pd
from model import ResNetModel
from tqdm import tqdm
sys.path.insert(0, '../utils')
from preprocessor import BatchPreprocessor

In [2]:
class ImageData:

    def __init__(self, batch_size, load_size, channels, augment_flag,num_class):
        self.batch_size = batch_size
        self.load_size = load_size
        self.channels = channels
        self.augment_flag = augment_flag
        self.num_class=num_class
        self.mean_color=tf.constant([132.2766, 139.6506, 146.9702],name="mean_color")

    def image_processing(self, filename,label):
        x = tf.read_file(filename)
        x_decode = tf.image.decode_jpeg(x, channels=self.channels)
        img = tf.image.resize_images(x_decode, [self.load_size, self.load_size])
        img = tf.cast(img, tf.float32) #/ 127.5 - 1

        if self.augment_flag :
            augment_size = self.load_size + (30 if self.load_size == 256 else 15)
            p = random.random()
            if p > 0.5:
                img = augmentation(img, augment_size)
                
        lab=tf.cast(label,tf.int32)
        lab=tf.one_hot(lab,self.num_class,dtype=tf.float32)
        #img=tf.subtract(img,self.mean_color)
        return img,lab


def augmentation(image, augment_size):
    seed = random.randint(0, 2 ** 31 - 1)
    ori_image_shape = tf.shape(image)
    image = tf.image.random_flip_left_right(image, seed=seed)
    image = tf.image.resize_images(image, [augment_size, augment_size])
    image = tf.random_crop(image, ori_image_shape, seed=seed)
    return image

In [31]:
checkpoint_dir="../training/resnet_20181022_150646/checkpoint"
#val_dir="/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test"
val_dir="/home/ye/user/yejg/database/ZocEye/origin/"
batch_size=64
num_classes=5

model_path=tf.train.latest_checkpoint(checkpoint_dir=checkpoint_dir)
val_file="../data/val.txt"

#### load evaluate dataset

In [32]:
val_preprocessor = BatchPreprocessor(dataset_file_path=val_file, num_classes=num_classes, output_size=[224, 224])
val_num_batches = np.floor(len(val_preprocessor.labels) / batch_size).astype(np.int16)

In [5]:
# Placeholders
x = tf.placeholder(tf.float32, [None,224, 224, 3])
y = tf.placeholder(tf.float32, [None,num_classes])
is_training = tf.placeholder('bool', [])


# Model
model = ResNetModel(is_training, depth=101, num_classes=num_classes)
model.inference(x)
# Training accuracy of the model
correct_pred = tf.equal(tf.argmax(model.prob, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
prediction=tf.argmax(model.prob,1)

In [33]:
saver = tf.train.Saver()
with tf.Session() as sess:
    
    pred=[]
    true=[]
    sess.run(tf.global_variables_initializer())
    #sess.run(val_iterator.initializer)
    saver.restore(sess=sess,save_path=model_path)
    
    print("{} Start validation".format(datetime.datetime.now()))
    for step in range(val_num_batches):
        #batch_x,batch_y=sess.run([val_img,val_lab])
        batch_x, batch_y = val_preprocessor.next_batch(batch_size)
        val_acc,val_pred=sess.run([accuracy,prediction],feed_dict={x:batch_x,y:batch_y,is_training:False})
        #val_pred=sess.run([accuracy,prediction],feed_dict={x:batch_x,y:batch_y,is_training:False})
        true.append(batch_y[0])
        pred.append(val_pred[0])
        
        if step%20==0:
           print("{} Validation Accuracy = {:.4f}".format(datetime.datetime.now(), val_acc))
            
            

INFO:tensorflow:Restoring parameters from ../training/resnet_20181022_150646/checkpoint/model_epoch100.ckpt
2018-11-13 14:58:38.969899 Start validation
2018-11-13 14:59:04.212095 Validation Accuracy = 0.9688
2018-11-13 15:05:27.439315 Validation Accuracy = 0.9844
2018-11-13 15:11:18.207774 Validation Accuracy = 0.8906
2018-11-13 15:16:06.991086 Validation Accuracy = 0.9062
2018-11-13 15:21:38.231008 Validation Accuracy = 0.8750
2018-11-13 15:27:14.577092 Validation Accuracy = 0.8750
